<a href="https://colab.research.google.com/github/ashwinsapre/GANfaces_iCarl/blob/main/iCarl_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow import keras
import keras.layers as L
from keras.datasets import fashion_mnist
from keras.applications import DenseNet121
from keras.applications import VGG16
import matplotlib.pyplot as plt

In [12]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


 For multi-task learning, branching NN out
- one branch predicts the class of FashionMNIST object (classification)
- other branch predicts whether the object is a "top" or not (0/1 output)

In [13]:
def create_model(n_classes, input_dim, cl_weight, b_weight, lr):
    '''
        Creating categorical classification model
    '''
    
    inputs=L.Input((input_dim,input_dim,1))
    
    xc=L.Conv2D(64, kernel_size=3, padding='same', strides=1)(inputs)
    xc=L.LeakyReLU(0.2)(xc)
    xc=L.Conv2D(64, kernel_size=3, padding='same', strides=1)(xc)
    xc=L.LeakyReLU(0.2)(xc)
    xc=L.MaxPool2D(pool_size=2, strides=2)(xc)
    
    xc=L.Conv2D(32, kernel_size=3, padding='same', strides=1)(inputs)
    xc=L.LeakyReLU(0.2)(xc)
    xc=L.Conv2D(32, kernel_size=3, padding='same', strides=1)(xc)
    xc=L.LeakyReLU(0.2)(xc)
    xc=L.MaxPool2D(pool_size=2, strides=2)(xc)
    
    xc=L.Conv2D(16, kernel_size=3, padding='same', strides=1)(xc)
    xc=L.LeakyReLU(0.2)(xc)
    xc=L.Conv2D(16, kernel_size=3, padding='same', strides=1)(xc)
    xc=L.LeakyReLU(0.2)(xc)
    xc=L.MaxPool2D(pool_size=2, strides=2)(xc)
    
    xc=L.Flatten()(xc)
    xc=L.Dense(n_classes)(xc)
    outputc=L.Softmax(name='outputc')(xc)
    
    '''
        Creating binary classification model (a top/not a top)
    '''
    
    x=L.LeakyReLU(0.2)(inputs)
    x=L.Conv2D(32, kernel_size=3, padding='same', strides=1)(x)
    x=L.LeakyReLU(0.2)(x)
    x=L.MaxPool2D(pool_size=2, strides=2)(x)
    
    x=L.Conv2D(16, kernel_size=3, padding='same', strides=1)(x)
    x=L.LeakyReLU(0.2)(x)
    x=L.Conv2D(16, kernel_size=3, padding='same', strides=1)(x)
    x=L.LeakyReLU(0.2)(x)
    x=L.MaxPool2D(pool_size=2, strides=2)(x)
    
    x=L.Flatten()(x)
    outputb=L.Dense(1, activation='sigmoid', name='outputb')(x)
    #outputb=L.Activation('sigmoid', name="outputb")(x)

    '''
        Combining both models
    '''
    model=keras.Model(inputs=inputs, outputs=[outputc, outputb])
    losses={'outputc':'categorical_crossentropy', 'outputb':'binary_crossentropy'}
    loss_weights=[cl_weight, b_weight]
    model.compile(optimizer=keras.optimizers.Adam(lr), 
                      loss=losses,
                      loss_weights=loss_weights,
                      metrics=['accuracy'])
    
    return model

In [14]:
def train_model(model, n_epochs:int, x_train, y_trainc, y_trainb, validation_split):
    history=model.fit(x=x_train, 
                      y={'outputc':y_trainc, 'outputb': y_trainb}, 
                      epochs=n_epochs, 
                      validation_split=0.1,
                      shuffle=True,
                      verbose=2)
    return model

In [15]:
def calc_feature_vectors(model, images):
    #extract feature vector from a layer of ALREADY-TRAINED model
    #size of feature vector=784
    feature_model=keras.Model(inputs=model.inputs, outputs=model.get_layer("flatten").output)
    feature_vectors=feature_model.predict(images)
    norms=np.linalg.norm(feature_vectors, keepdims=True, axis=1)
    feature_vectors=feature_vectors/norms
    return feature_vectors

In [16]:
def create_exemplar_set(mem_size, n_classes, feature_vectors, labels, reconstruct=False):
    per_class=mem_size//n_classes

    class_vectors={}
    mean_class_vectors={}
    class_vectors_distances={}

    #init dicts
    for i in range(n_classes):
      class_vectors[i]=[]
      mean_class_vectors[i]=[]
      class_vectors_distances[i]=[]
    
    #vectors belonging to class i go in the list in key i of class_vectors
    for i in range(len(labels)):
      class_vectors[labels[i]].append(feature_vectors[i])

    #calculate mean class vectors by summing all vectors per class and diving by length
    for i in range(n_classes):
      mean_class_vectors[i]=np.sum(class_vectors[i], axis=0)/len(class_vectors[i])
    
    #calculating distances from mean
    for i in range(len(labels)):
      class_vectors_distances[labels[i]].append(np.linalg.norm(mean_class_vectors[labels[i]] - feature_vectors[i]))

    #sorting vectors by their corresponding distances from class means
    for i in range(n_classes):
      class_vectors_distances[i], class_vectors[i] = (list(t) for t in zip(*sorted(zip(class_vectors_distances[i], class_vectors[i]))))

    exemplars_x = []
    exemplars_y = []

    #choose 'per_class' number of vectors
    for i in range(n_classes):
      exemplars_x.append(class_vectors[i][:per_class])
      exemplars_y+= per_class*[i]

    return exemplars_x, exemplars_y

In [17]:
def classify(X, mean_class_vectors, n_classes):
    #image embedding created
    image_vector=calc_feature_vectors(model, X)
    #distance to all mean vectors calculated
    distances=[]
    indices=[x for x in range(n_classes)]
    #sort class values by their corresponding distances
    for i in range(n_classes):
      distances.append(np.linalg.norm(mean_class_vectors[i] - image_vector))
    distances, indices = (list(t) for t in zip(*sorted(zip(distances, indices))))
    #return the class with the least distance
    return indices[0]

In [18]:
def transform_outputs(y_train, y_test):
    labeldict={}
    labeldict[0]=1
    labeldict[1]=0
    labeldict[2]=1
    labeldict[3]=1
    labeldict[4]=1
    labeldict[5]=0
    labeldict[6]=1
    labeldict[7]=0
    labeldict[8]=0
    labeldict[9]=0
    
    y_trainb=[]
    y_testb=[]
    y_trainc=keras.utils.to_categorical(y_train)
    y_testc=keras.utils.to_categorical(y_test)
    
    for i in range(len(y_train)):
        y_trainb.append(labeldict[y_train[i]])
        
    for i in range(len(y_test)):
        y_testb.append(labeldict[y_test[i]])
    
    return y_trainc, y_testc, np.asarray(y_trainb), np.asarray(y_testb)

In [ ]:

(x_train, y_train), (x_test, y_test)=tf.keras.datasets.fashion_mnist.load_data()
x_train=x_train.reshape((x_train.shape[0], 28,28,1))
x_test=x_test.reshape((x_test.shape[0],28,28,1))
y_trainc, y_testc, y_trainb, y_testb=transform_outputs(y_train, y_test)
x_train=x_train/255.0
x_test=x_test/255.0

#np.savetxt("ytrain_transformed.csv", y_train, delimiter=',')
#np.savetxt("ytest_transformed.csv", y_test, delimiter=',')
    
#y_train=np.loadtxt(open("ytrain_transformed.csv", "rb"), delimiter=',')
#y_test=np.loadtxt(open("ytest_transformed.csv", "rb"), delimiter=',')
    
model=create_model(n_classes=10, input_dim=28, cl_weight=1.0, b_weight=1.5, lr=0.0005)
    
train_model(model, 50, x_train, y_trainc, y_trainb, validation_split=0.1)
model.save("trainedmodel.h5")
#-----------------------------------------------------------------------#

In [29]:
'''
    TESTING CELL
'''
x=model.layers[-3].output
x=L.Dense(11)(x)
new_outputc=L.Softmax(name='new_outputc')(x)
new_model=keras.Model(inputs=model.inputs,
                      outputs=[new_outputc, outputb])
print(new_model.summary())
losses={'outputc':'categorical_crossentropy', 'outputb':'binary_crossentropy'}
#loss_weights=[cl_weight, b_weight]
#model.compile(optimizer=keras.optimizers.Adam(lr), loss=losses, loss_weights=loss_weights, metrics=['accuracy'])

NameError: ignored